# 📦 Data Preparation for Depression Detection

## NeuroSense Project - DAIC-WOZ Dataset Preprocessing

This notebook prepares the DAIC-WOZ dataset for the depression detection pipeline.

### Pipeline Overview:
1. **Download** - Fetch participant ZIP files from DAIC-WOZ server
2. **Extract** - Extract AUDIO.wav and TRANSCRIPT.csv files
3. **Analyze** - Check audio metadata (sample rate, duration, etc.)
4. **Separate** - Extract participant-only audio (remove interviewer)
5. **Process** - Apply noise reduction and normalization
6. **Labels** - Combine train/dev/test labels into single file
7. **Segment** - Split audio into 10-second overlapping chunks

### Dataset Info:
- **Source**: DAIC-WOZ (Distress Analysis Interview Corpus)
- **Participants**: 300-492 (189 total, some missing)
- **Content**: Clinical interviews with virtual interviewer "Ellie"
- **Labels**: PHQ-8 depression scores (binary: 0=not depressed, 1=depressed)

### Output Files:
- `DATASET/DAIC-WOZ/RAW_DATA` - Raw audio and transcript files
- `DATASET/DAIC-WOZ/PROCESSED/` - Cleaned participant audio
- `DATASET/DAIC-WOZ/RAW_DATA/audio_raw/` - Full interview audio files (*_AUDIO.wav)
- `DATASET/DAIC-WOZ/RAW_DATA/transcripts_raw/` - Timestamped transcripts (*_TRANSCRIPT.csv)
- `DATASET/DAIC-WOZ/RAW_DATA/covarep_features/` - Acoustic features at 10ms intervals (*_COVAREP.csv)
- `DATASET/DAIC-WOZ/RAW_DATA/formant_features/` - Vocal tract resonance frequencies (*_FORMANT.csv)
- `DATASET/DAIC-WOZ/RAW_DATA/segments/audio/` - 10-second audio segments
- `DATASET/DAIC-WOZ/metadata/labels_all.csv` - Combined labels with train/dev/test splits

---

In [ ]:
!pip install noisereduce

## 1. Environment Setup

Mount Google Drive and configure paths.

In [1]:
# ============================================
# MOUNT GOOGLE DRIVE
# ============================================
from google.colab import drive
drive.mount('/content/drive')

# ============================================
# IMPORTS & CONFIGURATION
# ============================================
import zipfile
import requests
import os

# Base path for all project files
BASE_PATH = '/content/drive/MyDrive/Final Project/DataSet/DAIC-WOZ'

# Source folder (where ZIP files will be downloaded)
zip_folder = f'{BASE_PATH}/RAW_DATA/full_data_participants_zip'

# Output folders for each file type
output_folders = {
    'AUDIO': f'{BASE_PATH}/RAW_DATA/audio_raw',
    'TRANSCRIPT': f'{BASE_PATH}/RAW_DATA/transcripts_raw',
    'COVAREP': f'{BASE_PATH}/RAW_DATA/covarep_features_raw',
    'FORMANT': f'{BASE_PATH}/RAW_DATA/formant_features_raw',
}

# Create all output folders
for folder in output_folders.values():
    os.makedirs(folder, exist_ok=True)
    print(f"📁 Created/verified: {folder}")


Mounted at /content/drive
📁 Created/verified: /content/drive/MyDrive/Final Project/DataSet/DAIC-WOZ/RAW_DATA/audio_raw
📁 Created/verified: /content/drive/MyDrive/Final Project/DataSet/DAIC-WOZ/RAW_DATA/transcripts_raw
📁 Created/verified: /content/drive/MyDrive/Final Project/DataSet/DAIC-WOZ/RAW_DATA/covarep_features_raw
📁 Created/verified: /content/drive/MyDrive/Final Project/DataSet/DAIC-WOZ/RAW_DATA/formant_features_raw


## 2. Download DAIC-WOZ Dataset

Download participant ZIP files from the DAIC-WOZ server.

**Note:**
- Participants range from 300-492
- Some IDs are missing (342, 394, 398, 460)
- Each ZIP contains: audio, transcript, facial features, etc.

In [ ]:
# ============================================
# DOWNLOAD DAIC-WOZ PARTICIPANT FILES
# ============================================

# DAIC-WOZ dataset URL
base_url = "https://dcapswoz.ict.usc.edu/wwwdaicwoz/"

# Participant ID range
participant_ids = list(range(300, 493))

# Counters for summary
downloaded = 0
skipped = 0
failed = 0

print(f"Downloading {len(participant_ids)} participants...\n")

for pid in participant_ids:
    filename = f"{pid}_P.zip"
    url = base_url + filename
    output_path = os.path.join(zip_folder, filename)

    # Skip if file already exists
    if os.path.exists(output_path):
        print(f"⏭️ {filename} - exists")
        skipped += 1
        continue

    # Download file
    try:
        response = requests.get(url, stream=True)

        if response.status_code == 200:
            # Save file in chunks (memory efficient)
            with open(output_path, 'wb') as f:
                for chunk in response.iter_content(chunk_size=8192):
                    f.write(chunk)

            size_mb = os.path.getsize(output_path) / (1024*1024)
            print(f"✅ {filename} - {size_mb:.0f} MB")
            downloaded += 1
        else:
            # Participant not found (some IDs are missing)
            print(f"❌ {filename} - not found")
            failed += 1

    except Exception as e:
        print(f"❌ {filename} - {e}")
        failed += 1

# ============================================
# SUMMARY
# ============================================
print("\n" + "=" * 40)
print(f"✅ Downloaded: {downloaded}")
print(f"⏭️ Skipped (already exists): {skipped}")
print(f"❌ Failed/Not found: {failed}")
print("=" * 40)

## DAIC-WOZ File Descriptions

### Audio & Text
- `*_AUDIO.wav` - Full interview audio recording (participant + interviewer)
- `*_TRANSCRIPT.csv` - Timestamped transcript with speaker labels (Participant/Ellie)

### Acoustic Features (COVAREP)
- `*_COVAREP.csv` - Acoustic features extracted at 10ms intervals (100Hz), including:
  - F0 (pitch), VUV (voiced/unvoiced flag)
  - NAQ, QOQ (voice quality measures)
  - H1H2, PSP, MDQ, peakSlope (glottal features)
  - MCEP_0-24 (Mel cepstral coefficients)
  - HMPDM_0-24, HMPDD_0-12 (harmonic model features)
- `*_FORMANT.csv` - First 5 formants (vocal tract resonance frequencies)


In [ ]:
import zipfile
import os
import time

#============================================
# FILE TYPE DETECTION FUNCTION
# ============================================
def get_file_type(filename):
    """Determine file type and target folder based on filename"""
    filename_upper = filename.upper()

    if 'AUDIO' in filename_upper and filename_upper.endswith('.WAV'):
        return 'AUDIO'
    elif 'TRANSCRIPT' in filename_upper and filename_upper.endswith('.CSV'):
        return 'TRANSCRIPT'
    elif 'COVAREP' in filename_upper and filename_upper.endswith('.CSV'):
        return 'COVAREP'
    elif 'FORMANT' in filename_upper and filename_upper.endswith('.CSV'):
        return 'FORMANT'
    else:
        return None  # Skip unknown files

# ============================================
# EXTRACT FILES FROM ZIP
# ============================================
max_retries = 3

# Counters per file type
extracted_counts = {key: 0 for key in output_folders.keys()}
skipped_counts = {key: 0 for key in output_folders.keys()}
failed_files = []

# Get list of ZIP files
zip_files = sorted([f for f in os.listdir(zip_folder) if f.endswith('_P.zip')])
print(f"\n🔍 Found {len(zip_files)} ZIP files to process\n")

for idx, filename in enumerate(zip_files):
    zip_path = os.path.join(zip_folder, filename)
    participant_id = filename.replace('_P.zip', '')

    print(f"\n📦 [{idx+1}/{len(zip_files)}] Processing {filename}...")

    for attempt in range(max_retries):
        try:
            with zipfile.ZipFile(zip_path, 'r') as z:
                for file in z.namelist():
                    # Get just the filename (without folder path inside ZIP)
                    base_filename = os.path.basename(file)
                    if not base_filename:  # Skip directories
                        continue

                    # Determine file type
                    file_type = get_file_type(base_filename)

                    if file_type:
                        target_folder = output_folders[file_type]
                        output_path = os.path.join(target_folder, base_filename)

                        if os.path.exists(output_path):
                            print(f"  ⏭️ {base_filename} → {file_type} (exists)")
                            skipped_counts[file_type] += 1
                        else:
                            # Extract to temp, then move to correct folder
                            z.extract(file, BASE_PATH)
                            extracted_path = os.path.join(BASE_PATH, file)

                            # Move to target folder
                            os.rename(extracted_path, output_path)
                            print(f"  ✅ {base_filename} → {file_type}")
                            extracted_counts[file_type] += 1

            break  # Success - exit retry loop

        except (ConnectionAbortedError, OSError) as e:
            print(f"  ⚠️ Attempt {attempt+1}/{max_retries} failed: {e}")
            if attempt < max_retries - 1:
                time.sleep(5)
            else:
                failed_files.append(filename)

    # Pause every 5 files
    if (idx + 1) % 5 == 0:
        print(f"\n⏸️ Pausing briefly...")
        time.sleep(2)

# ============================================
# SUMMARY
# ============================================
print("\n" + "=" * 60)
print("📊 EXTRACTION SUMMARY")
print("=" * 60)

print("\n✅ Extracted files:")
for file_type, count in extracted_counts.items():
    if count > 0:
        print(f"   {file_type}: {count} files → {output_folders[file_type]}")

print("\n⏭️ Skipped (already exist):")
for file_type, count in skipped_counts.items():
    if count > 0:
        print(f"   {file_type}: {count} files")

if failed_files:
    print(f"\n❌ Failed ZIPs: {len(failed_files)}")
    for f in failed_files:
        print(f"   - {f}")

print("\n" + "=" * 60)


🔍 Found 189 ZIP files to process


📦 [1/189] Processing 300_P.zip...
  ⏭️ 300_AUDIO.wav → AUDIO (exists)
  ✅ 300_COVAREP.csv → COVAREP
  ✅ 300_FORMANT.csv → FORMANT
  ⏭️ 300_TRANSCRIPT.csv → TRANSCRIPT (exists)

📦 [2/189] Processing 301_P.zip...
  ⏭️ 301_AUDIO.wav → AUDIO (exists)
  ⚠️ Attempt 1/3 failed: [Errno 107] Transport endpoint is not connected
  ⏭️ 301_AUDIO.wav → AUDIO (exists)
  ✅ 301_COVAREP.csv → COVAREP
  ✅ 301_FORMANT.csv → FORMANT
  ⏭️ 301_TRANSCRIPT.csv → TRANSCRIPT (exists)

📦 [3/189] Processing 302_P.zip...
  ⏭️ 302_AUDIO.wav → AUDIO (exists)
  ✅ 302_COVAREP.csv → COVAREP
  ✅ 302_FORMANT.csv → FORMANT
  ⏭️ 302_TRANSCRIPT.csv → TRANSCRIPT (exists)

📦 [4/189] Processing 303_P.zip...
  ⏭️ 303_AUDIO.wav → AUDIO (exists)
  ✅ 303_COVAREP.csv → COVAREP
  ✅ 303_FORMANT.csv → FORMANT
  ⏭️ 303_TRANSCRIPT.csv → TRANSCRIPT (exists)

📦 [5/189] Processing 304_P.zip...
  ⏭️ 304_AUDIO.wav → AUDIO (exists)
  ✅ 304_COVAREP.csv → COVAREP
  ✅ 304_FORMANT.csv → FORMANT
  ⏭️ 304_TRANSCRI

## 4. Analyze Audio Metadata

Collect metadata from all audio files:
- Sample rate
- Duration
- Number of channels
- File size

In [ ]:
# ============================================
# ANALYZE AUDIO FILE METADATA
# ============================================
import librosa
import os
import pandas as pd
import numpy as np

folder = f'{BASE_PATH}/RAW_DATA/audio_raw'

# Collect metadata for each audio file
data = []

audio_files = sorted([f for f in os.listdir(folder) if f.endswith("_AUDIO.wav")])
print(f"Found {len(audio_files)} audio files\n")

for filename in audio_files:
    file_path = os.path.join(folder, filename)
    participant_id = filename.replace("_AUDIO.wav", "")

    try:
        # Load audio file
        audio, sr = librosa.load(file_path, sr=None)

        # Get number of channels
        channels = 1 if len(audio.shape) == 1 else audio.shape[0]

        # Calculate duration
        duration_sec = len(audio) / sr
        duration_min = duration_sec / 60

        data.append({
            "participant_id": participant_id,
            "sample_rate": sr,
            "channels": channels,
            "duration_sec": round(duration_sec, 1),
            "duration_min": round(duration_min, 2),
            "file_size_mb": round(os.path.getsize(file_path) / (1024*1024), 1),
            "status": "✅ OK"
        })
        print(f"✅ {participant_id}")

    except Exception as e:
        data.append({
            "participant_id": participant_id,
            "sample_rate": None,
            "channels": None,
            "duration_sec": None,
            "duration_min": None,
            "file_size_mb": None,
            "status": f"❌ {e}"
        })
        print(f"❌ {participant_id}: {e}")

# Create DataFrame
df = pd.DataFrame(data)

# ============================================
# DISPLAY RESULTS
# ============================================
print("\n" + "=" * 60)
print("AUDIO FILES METADATA")
print("=" * 60)
display(df)

# ============================================
# STATISTICS
# ============================================
print("\n" + "=" * 60)
print("STATISTICS")
print("=" * 60)

df_valid = df[df["status"] == "✅ OK"]

print(f"Total Files: {len(df)}")
print(f"Valid Files: {len(df_valid)}")
print(f"Failed Files: {len(df) - len(df_valid)}")
print(f"\nSample Rates: {df_valid['sample_rate'].unique().tolist()}")
print(f"Channels: {df_valid['channels'].unique().tolist()}")
print(f"\nDuration (min):")
print(f"  Min: {df_valid['duration_min'].min()}")
print(f"  Max: {df_valid['duration_min'].max()}")
print(f"  Mean: {round(df_valid['duration_min'].mean(), 2)}")
print(f"  Std: {round(df_valid['duration_min'].std(), 2)}")
print(f"\nTotal Duration: {round(df_valid['duration_min'].sum() / 60, 2)} hours")
print(f"Total Size: {round(df_valid['file_size_mb'].sum() / 1024, 2)} GB")

# Save metadata
output_path = f"{BASE_PATH}/RAW_DATA/audio_raw/audio_metadata.csv"
df.to_csv(output_path, index=False)
print(f"\n✅ Saved to: {output_path}")

Found 189 audio files

✅ 300
✅ 301
✅ 302
✅ 303
✅ 304
✅ 305
✅ 306
✅ 307
✅ 308
✅ 309
✅ 310
✅ 311
✅ 312
✅ 313
✅ 314
✅ 315
✅ 316
✅ 317
✅ 318
✅ 319
✅ 320
✅ 321
✅ 322
✅ 323
✅ 324
✅ 325
✅ 326
✅ 327
✅ 328
✅ 329
✅ 330
✅ 331
✅ 332
✅ 333
✅ 334
✅ 335
✅ 336
✅ 337
✅ 338
✅ 339
✅ 340
✅ 341
✅ 343
✅ 344
✅ 345
✅ 346
✅ 347
✅ 348
✅ 349
✅ 350
✅ 351
✅ 352
✅ 353
✅ 354
✅ 355
✅ 356
✅ 357
✅ 358
✅ 359
✅ 360
✅ 361
✅ 362
✅ 363
✅ 364
✅ 365
✅ 366
✅ 367
✅ 368
✅ 369
✅ 370
✅ 371
✅ 372
✅ 373
✅ 374
✅ 375
✅ 376
✅ 377
✅ 378
✅ 379
✅ 380
✅ 381
✅ 382
✅ 383
✅ 384
✅ 385
✅ 386
✅ 387
✅ 388
✅ 389
✅ 390
✅ 391
✅ 392
✅ 393
✅ 395
✅ 396
✅ 397
✅ 399
✅ 400
✅ 401
✅ 402
✅ 403
✅ 404
✅ 405
✅ 406
✅ 407
✅ 408
✅ 409
✅ 410
✅ 411
✅ 412
✅ 413
✅ 414
✅ 415
✅ 416
✅ 417
✅ 418
✅ 419
✅ 420
✅ 421
✅ 422
✅ 423
✅ 424
✅ 425
✅ 426
✅ 427
✅ 428
✅ 429
✅ 430
✅ 431
✅ 432
✅ 433
✅ 434
✅ 435
✅ 436
✅ 437
✅ 438
✅ 439
✅ 440
✅ 441
✅ 442
✅ 443
✅ 444
✅ 445
✅ 446
✅ 447
✅ 448
✅ 449
✅ 450
✅ 451
✅ 452
✅ 453
✅ 454
✅ 455
✅ 456
✅ 457
✅ 458
✅ 459
✅ 461
✅ 462
✅ 463
✅ 464
✅ 465
✅ 466

,participant_id,sample_rate,channels,duration_sec,duration_min,file_size_mb,status
0,300,16000,1,648.5,10.81,19.8,✅ OK
1,301,16000,1,823.9,13.73,25.1,✅ OK
2,302,16000,1,758.8,12.65,23.2,✅ OK
3,303,16000,1,985.3,16.42,30.1,✅ OK
4,304,16000,1,792.6,13.21,24.2,✅ OK
...,...,...,...,...,...,...,...
184,488,16000,1,884.9,14.75,27.0,✅ OK
185,489,16000,1,704.7,11.75,21.5,✅ OK
186,490,16000,1,691.3,11.52,21.1,✅ OK
187,491,16000,1,881.7,14.70,26.9,✅ OK



STATISTICS
Total Files: 189
Valid Files: 189
Failed Files: 0

Sample Rates: [16000]
Channels: [1]

Duration (min):
  Min: 6.91
  Max: 32.77
  Mean: 15.94
  Std: 4.5

Total Duration: 50.21 hours
Total Size: 5.39 GB

✅ Saved to: /content/drive/MyDrive/Final Project/DataSet/DAIC-WOZ/RAW_DATA/audio_raw/audio_metadata.csv


## 5. Extract Participant Audio Only

Extract only the participant's speech from the interview:
- Uses transcript timestamps to identify participant segments
- Preserves pauses between participant utterances
- Removes interviewer ("Ellie") audio

**Why?** The model should learn from participant's speech patterns, not the interviewer's questions.

In [ ]:
# ============================================
# EXTRACT PARTICIPANT-ONLY AUDIO
# ============================================
import pandas as pd
import librosa
import numpy as np
import soundfile as sf
import os

def extract_participant_audio_full(audio_path, transcript_path):
    """
    Extract participant's audio segments from the full interview.

    Preserves:
    - Response latency (pause before responding)
    - Pauses within participant's speech
    - Pauses between sentences

    Args:
        audio_path: Path to full interview audio
        transcript_path: Path to transcript CSV with timestamps

    Returns:
        participant_audio: Concatenated participant audio
        sr: Sample rate
    """
    # Load audio
    audio, sr = librosa.load(audio_path, sr=None)

    # Load transcript (tab-separated)
    df = pd.read_csv(transcript_path, sep='\t')

    segments = []
    i = 0

    while i < len(df):
        # Find start of participant block
        if df.iloc[i]['speaker'] == 'Participant':

            # Get pause start (end of previous speaker)
            if i > 0:
                block_start = df.iloc[i-1]['stop_time']
            else:
                block_start = df.iloc[i]['start_time']

            # Find end of participant block (consecutive participant rows)
            block_end = df.iloc[i]['stop_time']

            while i + 1 < len(df) and df.iloc[i + 1]['speaker'] == 'Participant':
                i += 1
                block_end = df.iloc[i]['stop_time']

            # Extract audio block
            start_sample = int(block_start * sr)
            stop_sample = int(block_end * sr)
            segment = audio[start_sample:stop_sample]
            segments.append(segment)

        i += 1

    # Concatenate all participant segments
    if segments:
        participant_audio = np.concatenate(segments)
    else:
        participant_audio = np.array([])

    return participant_audio, sr


# ============================================
# PROCESS ALL FILES
# ============================================
audio_folder = f'{BASE_PATH}/RAW_DATA/audio_raw'
transcript_folder = f'{BASE_PATH}/RAW_DATA/transcripts_raw'
output_folder = f"{BASE_PATH}/RAW_DATA/audio_raw_only_patient"

os.makedirs(output_folder, exist_ok=True)

for filename in sorted(os.listdir(audio_folder)):
    if filename.endswith("_AUDIO.wav"):
        participant_id = filename.replace("_AUDIO.wav", "")

        audio_path = os.path.join(audio_folder, filename)
        transcript_path = os.path.join(transcript_folder, f"{participant_id}_TRANSCRIPT.csv")

        if os.path.exists(transcript_path):
            # Extract participant audio
            audio, sr = extract_participant_audio_full(audio_path, transcript_path)

            # Save participant-only audio
            output_path = os.path.join(output_folder, f"{participant_id}_PARTICIPANT.wav")
            sf.write(output_path, audio, sr)

            duration = len(audio) / sr
            print(f"✅ {participant_id}: {duration:.1f}s of participant audio")
        else:
            print(f"❌ {participant_id}: transcript not found")

print("\n✅ Done!")

✅ 300: 272.8s of participant audio
✅ 301: 573.0s of participant audio
✅ 302: 376.8s of participant audio
✅ 303: 738.6s of participant audio
✅ 304: 453.5s of participant audio
✅ 305: 1420.6s of participant audio
✅ 306: 558.8s of participant audio
✅ 307: 1003.0s of participant audio
✅ 308: 604.8s of participant audio
✅ 309: 331.2s of participant audio
✅ 310: 529.0s of participant audio
✅ 311: 392.0s of participant audio
✅ 312: 489.3s of participant audio
✅ 313: 437.9s of participant audio
✅ 314: 1267.0s of participant audio
✅ 315: 635.8s of participant audio
✅ 316: 467.7s of participant audio
✅ 317: 473.0s of participant audio
✅ 318: 310.0s of participant audio
✅ 319: 353.0s of participant audio
✅ 320: 410.5s of participant audio
✅ 321: 433.5s of participant audio
✅ 322: 721.4s of participant audio
✅ 323: 528.7s of participant audio
✅ 324: 394.6s of participant audio
✅ 325: 621.1s of participant audio
✅ 326: 305.0s of participant audio
✅ 327: 334.0s of participant audio
✅ 328: 804.2s of 

## 6. Audio Processing

Apply audio preprocessing:
1. **DC Offset Removal** - Subtract mean to center signal
2. **High-pass Filter** - Remove frequencies below 80 Hz (noise, not speech)
3. **Noise Reduction** (optional) - Remove background noise
4. **Normalization** - Scale to [-1, 1] range

In [ ]:
# Install noise reduction library
!pip install noisereduce -q

In [ ]:
# ============================================
# AUDIO PREPROCESSING
# ============================================
import librosa
import numpy as np
import soundfile as sf
import noisereduce as nr
from scipy.signal import butter, filtfilt
import os

# Configuration
input_folder = f"{BASE_PATH}/RAW_DATA/audio_raw_only_patient/"
output_folder = f"{BASE_PATH}/clean_audio/"
os.makedirs(output_folder, exist_ok=True)

# Processing options
APPLY_NOISE_REDUCTION = False  # Set to True if audio quality is poor


def highpass_filter(audio, sr, cutoff=80):
    """
    Remove frequencies below cutoff (default 80 Hz).

    Low frequencies are typically noise, not human speech.
    Human speech fundamental frequency: ~85-255 Hz

    Args:
        audio: Audio signal
        sr: Sample rate
        cutoff: Cutoff frequency in Hz

    Returns:
        Filtered audio signal
    """
    nyquist = sr / 2
    normalized_cutoff = cutoff / nyquist
    b, a = butter(5, normalized_cutoff, btype='high')
    filtered_audio = filtfilt(b, a, audio)
    return filtered_audio


# ============================================
# PROCESS ALL AUDIO FILES
# ============================================
for filename in sorted(os.listdir(input_folder)):
    if filename.endswith(".wav"):

        # Load audio file (keep original sample rate)
        audio, sr = librosa.load(os.path.join(input_folder, filename), sr=None)

        # Step 1: Remove DC Offset (subtract mean)
        audio = audio - np.mean(audio)

        # Step 2: Apply high-pass filter (remove frequencies below 80 Hz)
        audio = highpass_filter(audio, sr, cutoff=80)

        # Step 3 (Optional): Apply noise reduction
        if APPLY_NOISE_REDUCTION:
            audio = nr.reduce_noise(y=audio, sr=sr)

        # Step 4: Normalize to [-1, 1] range
        audio = audio / np.max(np.abs(audio))

        # Save processed audio
        output_filename = filename.replace(".wav", "_clean.wav")
        output_path = os.path.join(output_folder, output_filename)
        sf.write(output_path, audio, sr)

        print(f"✅ {output_filename}")

print("\n✅ Audio processing complete!")

✅ 300_PARTICIPANT_clean.wav
✅ 301_PARTICIPANT_clean.wav
✅ 302_PARTICIPANT_clean.wav
✅ 303_PARTICIPANT_clean.wav
✅ 304_PARTICIPANT_clean.wav
✅ 305_PARTICIPANT_clean.wav
✅ 306_PARTICIPANT_clean.wav
✅ 307_PARTICIPANT_clean.wav
✅ 308_PARTICIPANT_clean.wav
✅ 309_PARTICIPANT_clean.wav
✅ 310_PARTICIPANT_clean.wav
✅ 311_PARTICIPANT_clean.wav
✅ 312_PARTICIPANT_clean.wav
✅ 313_PARTICIPANT_clean.wav
✅ 314_PARTICIPANT_clean.wav
✅ 315_PARTICIPANT_clean.wav
✅ 316_PARTICIPANT_clean.wav
✅ 317_PARTICIPANT_clean.wav
✅ 318_PARTICIPANT_clean.wav
✅ 319_PARTICIPANT_clean.wav
✅ 320_PARTICIPANT_clean.wav
✅ 321_PARTICIPANT_clean.wav
✅ 322_PARTICIPANT_clean.wav
✅ 323_PARTICIPANT_clean.wav
✅ 324_PARTICIPANT_clean.wav
✅ 325_PARTICIPANT_clean.wav
✅ 326_PARTICIPANT_clean.wav
✅ 327_PARTICIPANT_clean.wav
✅ 328_PARTICIPANT_clean.wav
✅ 329_PARTICIPANT_clean.wav
✅ 330_PARTICIPANT_clean.wav
✅ 331_PARTICIPANT_clean.wav
✅ 332_PARTICIPANT_clean.wav
✅ 333_PARTICIPANT_clean.wav
✅ 334_PARTICIPANT_clean.wav
✅ 335_PARTICIPANT_cl

## 7. Create Combined Labels File

Combine train/dev/test label files into a single `labels_all.csv`:
- Standardize column names (PHQ8_Binary → PHQ_Binary)
- Add split column (train/dev/test)
- Display class distribution

In [2]:
# ============================================
# COMBINE LABEL FILES (CORRECTED FOR REGRESSION)
# ============================================
import pandas as pd
import os

folder = BASE_PATH + '/RAW_DATA/train_dev_test'

# Load the 3 split files
print("⏳ Loading split files...")
train = pd.read_csv(os.path.join(folder, "train_split_Depression_AVEC2017.csv"))
dev = pd.read_csv(os.path.join(folder, "dev_split_Depression_AVEC2017.csv"))
test = pd.read_csv(os.path.join(folder, "full_test_split.csv"))

# ---------------------------------------------------------
# 1. Standardize Column Names (Binary AND Score)
# ---------------------------------------------------------
# Binary
if 'PHQ8_Binary' in train.columns:
    train = train.rename(columns={'PHQ8_Binary': 'PHQ_Binary'})
if 'PHQ8_Binary' in dev.columns:
    dev = dev.rename(columns={'PHQ8_Binary': 'PHQ_Binary'})

# Score (החלק החדש והחשוב לרגרסיה)
if 'PHQ8_Score' in train.columns:
    train = train.rename(columns={'PHQ8_Score': 'PHQ_Score'})
if 'PHQ8_Score' in dev.columns:
    dev = dev.rename(columns={'PHQ8_Score': 'PHQ_Score'})

# Test usually has 'PHQ_Score', but let's be safe
if 'PHQ8_Score' in test.columns:
    test = test.rename(columns={'PHQ8_Score': 'PHQ_Score'})

# ---------------------------------------------------------
# 2. Verify Columns
# ---------------------------------------------------------
print("=" * 50)
print("COLUMN VERIFICATION")
print("=" * 50)
print(f"Train: Binary? {'✅' if 'PHQ_Binary' in train.columns else '❌'} | Score? {'✅' if 'PHQ_Score' in train.columns else '❌'}")
print(f"Dev:   Binary? {'✅' if 'PHQ_Binary' in dev.columns else '❌'} | Score? {'✅' if 'PHQ_Score' in dev.columns else '❌'}")
print(f"Test:  Binary? {'✅' if 'PHQ_Binary' in test.columns else '❌'} | Score? {'✅' if 'PHQ_Score' in test.columns else '❌'}")

# ============================================
# COMBINE ALL SPLITS
# ============================================
train['split'] = 'train'
dev['split'] = 'dev'
test['split'] = 'test'

combined = pd.concat([train, dev, test], ignore_index=True)

# ============================================
# SAVE COMBINED FILE (WITH SCORE)
# ============================================
output_path = os.path.join(BASE_PATH, "DataSet/DAIC_WOZ/metadata/labels_all.csv")

# מוודאים שהתיקייה קיימת
os.makedirs(os.path.dirname(output_path), exist_ok=True)

# כאן השינוי החשוב: שומרים גם את PHQ_Score
cols_to_save = ['Participant_ID', 'PHQ_Binary', 'PHQ_Score', 'split']

# בדיקה אחרונה שלא חסר כלום לפני שמירה
final_cols = [c for c in cols_to_save if c in combined.columns]
combined[final_cols].to_csv(output_path, index=False)

print("\n" + "=" * 50)
print(f"✅ DONE! Saved labels with scores to:")
print(f"{output_path}")
print("=" * 50)
print("First 5 rows:")
print(combined[final_cols].head())

⏳ Loading split files...
COLUMN VERIFICATION
Train: Binary? ✅ | Score? ✅
Dev:   Binary? ✅ | Score? ✅
Test:  Binary? ✅ | Score? ✅

✅ DONE! Saved labels with scores to:
/content/drive/MyDrive/Final Project/DataSet/DAIC-WOZ/DataSet/DAIC_WOZ/metadata/labels_all.csv
First 5 rows:
   Participant_ID  PHQ_Binary  PHQ_Score  split
0             303           0          0  train
1             304           0          6  train
2             305           0          7  train
3             310           0          4  train
4             312           0          2  train


## 8. Segment Audio Files

Split processed audio into fixed-length segments:
- **Segment length**: 10 seconds
- **Overlap**: 5 seconds (50%)
- **Last segment**: Zero-padded if > 2 seconds

**Why segment?**
- Wav2Vec2 works best with shorter clips
- Enables data augmentation through different segments
- Reduces memory requirements during training

In [ ]:
# ============================================
# SEGMENT AUDIO FILES
# ============================================
import librosa
import numpy as np
import soundfile as sf
import pandas as pd
import os

# Configuration
input_folder = f"{BASE_PATH}/clean_audio/"
output_folder = f"{BASE_PATH}/segments/audio/"
os.makedirs(output_folder, exist_ok=True)

# Segmentation parameters
SEGMENT_SEC = 10         # Segment length in seconds
OVERLAP_SEC = 5          # Overlap between segments
MIN_LAST_SEGMENT_SEC = 2 # Minimum length to keep last segment

print(f"Segmentation settings:")
print(f"  Segment length: {SEGMENT_SEC}s")
print(f"  Overlap: {OVERLAP_SEC}s")
print(f"  Hop size: {SEGMENT_SEC - OVERLAP_SEC}s")
print(f"  Min last segment: {MIN_LAST_SEGMENT_SEC}s\n")

# ============================================
# PROCESS ALL FILES
# ============================================
results = []

for filename in sorted(os.listdir(input_folder)):
    if filename.endswith("_PARTICIPANT_clean.wav"):
        participant_id = filename.replace("_PARTICIPANT_clean.wav", "")

        # Load processed audio
        audio, sr = librosa.load(os.path.join(input_folder, filename), sr=None)

        # Calculate segment parameters in samples
        segment_samples = int(SEGMENT_SEC * sr)
        hop_samples = int((SEGMENT_SEC - OVERLAP_SEC) * sr)
        min_last_samples = int(MIN_LAST_SEGMENT_SEC * sr)

        seg_count = 0

        # Create full segments
        for start in range(0, len(audio) - segment_samples + 1, hop_samples):
            segment = audio[start:start + segment_samples]
            output_path = os.path.join(output_folder, f"{participant_id}_seg{seg_count:03d}.wav")
            sf.write(output_path, segment, sr)
            seg_count += 1
            last_end = start + segment_samples

        # Handle remainder
        if seg_count > 0:
            remainder = audio[last_end:]
        else:
            remainder = audio

        # Keep last segment if long enough (pad with zeros)
        if len(remainder) >= min_last_samples:
            padded = np.zeros(segment_samples)
            padded[:len(remainder)] = remainder
            output_path = os.path.join(output_folder, f"{participant_id}_seg{seg_count:03d}.wav")
            sf.write(output_path, padded, sr)
            seg_count += 1

        results.append({'participant_id': participant_id, 'num_segments': seg_count})
        print(f"✅ {participant_id}: {seg_count} segments")

# ============================================
# SUMMARY
# ============================================
results_df = pd.DataFrame(results)
print(f"\n" + "=" * 50)
print(f"SUMMARY")
print("=" * 50)
print(f"Total participants: {len(results_df)}")
print(f"Total segments: {results_df['num_segments'].sum()}")
print(f"Segments per participant:")
print(f"  Min: {results_df['num_segments'].min()}")
print(f"  Max: {results_df['num_segments'].max()}")
print(f"  Mean: {results_df['num_segments'].mean():.1f}")

# Save metadata
results_df.to_csv(os.path.join(output_folder, "segments_metadata.csv"), index=False)
print(f"\n✅ Metadata saved to: segments_metadata.csv")

Segmentation settings:
  Segment length: 10s
  Overlap: 5s
  Hop size: 5s
  Min last segment: 2s

✅ 305: 283 segments
✅ 306: 111 segments
✅ 307: 200 segments
✅ 308: 120 segments
✅ 309: 65 segments
✅ 310: 105 segments
✅ 311: 77 segments
✅ 312: 97 segments
✅ 313: 87 segments
✅ 314: 253 segments
✅ 315: 126 segments
✅ 316: 93 segments
✅ 317: 94 segments
✅ 318: 61 segments
✅ 319: 70 segments
✅ 320: 81 segments
✅ 321: 86 segments
✅ 322: 143 segments
✅ 323: 105 segments
✅ 324: 78 segments
✅ 325: 123 segments
✅ 326: 60 segments
✅ 327: 66 segments
✅ 328: 160 segments
✅ 329: 83 segments
✅ 330: 87 segments
✅ 331: 107 segments
✅ 332: 101 segments
✅ 333: 122 segments
✅ 334: 121 segments
✅ 335: 119 segments
✅ 336: 112 segments
✅ 337: 335 segments
✅ 338: 67 segments
✅ 339: 112 segments
✅ 340: 55 segments
✅ 341: 115 segments
✅ 343: 90 segments
✅ 344: 151 segments
✅ 345: 114 segments
✅ 346: 197 segments
✅ 347: 54 segments
✅ 348: 79 segments
✅ 349: 175 segments
✅ 350: 120 segments
✅ 351: 99 segments
✅ 3



### Dataset Statistics:
| Split | Participants | Depressed | Not Depressed |
|-------|--------------|-----------|---------------|
| Train | 107 | 30 | 77 |
| Dev | 35 | 12 | 23 |
| Test | 47 | 12 | 35 |
| **Total** | **189** | **54** | **135** |

### Next Steps:
1. Run `01_audio.ipynb` to extract Wav2Vec2 embeddings
2. Run `02_text.ipynb` to train BERT on transcripts
3. Run `03_fusion.ipynb` to combine modalities

---